In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
from matplotlib.image import imread
import tensorflow_datasets as tfds
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator


print("Pandas Version %s" %pd.__version__)
print("Numpy version %s" % np.__version__)
print("Tensorflow Version %s" % tf.__version__)
print("CV Version %s" % cv.__version__)

Pandas Version 1.1.5
Numpy version 1.19.2
Tensorflow Version 2.1.0
CV Version 4.5.1


In [2]:
def preprocessImage(image):
    image = image / 255.0 
    image = cv.resize(image,(400,400))
    return image

In [3]:
# Take A look To data
print("Train Covid " ,len(os.listdir("Data/DataSetXray/train/COVID19/")))
print("Train normal", len(os.listdir("Data/DataSetXray/train/NORMAL//")))
print("Train Pneumonia", len(os.listdir("Data/DataSetXray/train/PNEUMONIA//")))


print("Test Covid", len(os.listdir("Data/DataSetXray/test/COVID19/")))
print("Test Normal", len(os.listdir("Data/DataSetXray/test/NORMAL//")))
print("Test PneuMonia" ,len(os.listdir("Data/DataSetXray/test/PNEUMONIA//")))

print("Test Covid" ,len(os.listdir("Data/DataSetCT/test/covid/")))
print("Test Non covid" ,len(os.listdir("Data/DataSetCT/test/noncovid//")))
print("Train covid" ,len(os.listdir("Data/DataSetCT/train/covid/")))
print("Train Non Covid", len(os.listdir("Data/DataSetCT/train/noncovid//")))


Train Covid  460
Train normal 1266
Train Pneumonia 3418
Test Covid 116
Test Normal 317
Test PneuMonia 855
Test Covid 100
Test Non covid 100
Train covid 1252
Train Non Covid 1229


In [4]:
TRAININGXRAY = "Data/DataSetXray/train/"
TESTXRAY = "Data/DataSetXray/test/"

TRAININGCT = "Data/DataSetCT/train/"
TESTCT = "Data/DataSetCT/test/"



In [5]:
trainXrayDatagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode="nearest")

trainXrayGenerator = trainXrayDatagen.flow_from_directory(TRAININGXRAY,
                                                         batch_size=32,
                                                         class_mode="categorical",
                                                         target_size=(150,150))

testXrayDatagen = ImageDataGenerator(rescale=1./255,)
testXrayGenerator = testXrayDatagen.flow_from_directory(TESTXRAY,
                                                       batch_size=32,
                                                       class_mode="categorical",
                                                       target_size=(150,150))

trainCtDatagen = ImageDataGenerator(rescale=  1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range= 0.2,
                                   shear_range = 0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest")
trainCtGenerator = trainCtDatagen.flow_from_directory(TRAININGCT,
                                                     batch_size=32,
                                                     class_mode="binary",
                                                     target_size=(150,150))
testCtDatagen = ImageDataGenerator(rescale=1./255)
testCtGenerator = testCtDatagen.flow_from_directory(TESTCT,
                                                   batch_size=32,
                                                   class_mode="binary",
                                                   target_size=(150,150))




Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.
Found 2481 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
# Creating Model For Both using modeSubclassing
class XrayModel(tf.keras.Model):
    
    def __init__(self, filters =  64, kernel_size = 7):
        super(XrayModel,self).__init__()
        
        self.convolution = tf.keras.layers.Conv2D(filters,kernel_size=kernel_size,padding="SAME")
        self.maxPooling = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.batchNormalization = tf.keras.layers.BatchNormalization()
        self.convolution2 = tf.keras.layers.Conv2D( filters, (3,3), padding= "SAME")
        self.relu = tf.keras.layers.ReLU()
        self.concat = tf.keras.layers.Concatenate()
        self.globalAvgPooling = tf.keras.layers.GlobalAveragePooling2D()
        self.flatten = tf.keras.layers.Flatten()
        
        self.dense = tf.keras.layers.Dense(1024, activation=tf.nn.relu)
        self.outputLayer = tf.keras.layers.Dense(3, activation = tf.nn.softmax)
        
    def call(self, input_tensors):
        
        layers = self.convolution(input_tensors)
        layers = self.relu(layers)
        layers = self.maxPooling(layers)
        layers = self.batchNormalization(layers)
        layers = self.globalAvgPooling(layers)
        
        layers2 = self.convolution2(input_tensors)
        layers2 = self.relu(layers2)
        layers2 = self.maxPooling(layers2)
        layers2 = self.batchNormalization(layers2)
        layers2 = self.globalAvgPooling(layers2)
        
        finLayer = self.concat([layers,layers2])

        dense = self.dense(finLayer)
        output = self.outputLayer(dense)
        
        return output
xrayModel = XrayModel() 
xrayModel.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                 loss ="categorical_crossentropy", 
                 metrics=["acc"])

xrayModel.fit(trainXrayGenerator,validation_data=testXrayGenerator)

  ...
    to  
  ['...']
Train for 161 steps, validate for 41 steps
 15/161 [=>............................] - ETA: 9:39 - loss: 0.9437 - acc: 0.6521

In [67]:
# Creating ResNet Architecthure But not going to use it :

class identityBlockOne(tf.keras.Model):
    
    def __init__(self, filters : int, kernel_size : int):
        super(identityBlockOne,self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(filters,kernel_size,padding="SAME")
        self.batchNorm1 = tf.keras.layers.BatchNormalization()
        self.pooling1 = tf.keras.layers.MaxPool2D((3,3),padding="same")
    
        self.activation = tf.keras.layers.Activation(tf.nn.relu)
        self.add = tf.keras.layers.Add()
        
    def call (self, input_tensors):
        
        layer = self.conv1(input_tensors)
        for i in range(2):
            layer = self.conv1(layer)
            layer = self.activation(layer)
            layer = self.batchNorm1(layer)

        layer = self.add([layer,input_tensors])
        layer = self.activation(layer)
        return layer
    
class identityBlockTwo(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(identityBlockTwo,self).__init__()
        self.amountLoop = 5
        self.conv1 = tf.keras.layers.Conv2D(filters,kernel_size,padding="SAME")
        self.activation = tf.keras.layers.Activation(tf.nn.relu)
        self.pooling = tf.keras.layers.MaxPool2D((3,3))
        self.batchNormalization = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters,kernel_size)
        self.add = tf.keras.layers.Add()
    
    def call (self, input_tensors):

        layer = self.conv1(input_tensors)
        layer = self.activation(layer)
        layer = self.batchNormalization(layer)
        
        for i in range(self.amountLoop):
            layer = self.conv1(layer)
            layer = self.activation(layer)
            layer = self.batchNormalization(layer)
            
        layer = self.add([layer,input_tensors])
        layer = self.activation(layer)
        
        return layer


class ResNet(tf.keras.Model):
    
    def __init__(self, num_classes):
        super(ResNet,self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(64, 7, padding="SAME")
        self.batchNormalization = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.Activation(tf.nn.relu)
        self.max_pool = tf.keras.layers.MaxPool2D((3,3))
        
        self.id1 = identityBlockOne(64, 5)
        self.id2 = identityBlockTwo(64 ,5)
        
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation = tf.nn.softmax)
    
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batchNormalization(x)
        x = self.activation(x)
        x = self.max_pool(x)
        
        x = self.id1(x)
        for i in range(0,3):
            x = self.id2(x)
        
        x = self.global_pool(x)
        
        return self.classifier(x)

In [69]:
def testPreprocess(features):
    return tf.cast(features["image"], tf.float32)/ 255, features['label'] 
resnet = ResNet(10)
resnet.compile('adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])

dataset = tfds.load('mnist',split=tfds.Split.TRAIN)
dataset = dataset.map(testPreprocess).batch(32)

history = resnet.fit(dataset,epochs=1)
print(history.history["accuracy"])

   1807/Unknown - 1347s 746ms/step - loss: 0.1578 - accuracy: 0.9525

KeyboardInterrupt: 